In [0]:
import tensorflow as tf
import numpy as np
import os
import time

In [21]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
path_to_file = '/content/drive/My Drive/data_1.txt'

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [0]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 91) # (batch_size, sequence_length, vocab_size)


In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           23296     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 91)            93275     
Total params: 4,054,875
Trainable params: 4,054,875
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [35]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'vestment opportunities and strengthen \nprotection for investments and investment \nactivities in the '

Next Char Predictions: 
 'n–Y26–s)ív/qD“0)crjvGbpvKFb4JJo’mSm晋Pks’ó–;üüuK–9sZnb7ÜbÜa.9YILPIrBZgs”só9G,0z3a晋安“Ekúo”i-a\t晋dxúrke”'


In [36]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 91)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.511845


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [39]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
64/64 [==============================] - 317s 5s/step - loss: 3.1597
Epoch 2/10
64/64 [==============================] - 316s 5s/step - loss: 2.2192
Epoch 3/10
64/64 [==============================] - 320s 5s/step - loss: 1.8650
Epoch 4/10
64/64 [==============================] - 312s 5s/step - loss: 1.5133
Epoch 5/10
64/64 [==============================] - 315s 5s/step - loss: 1.2657
Epoch 6/10
64/64 [==============================] - 326s 5s/step - loss: 1.1128
Epoch 7/10
64/64 [==============================] - 323s 5s/step - loss: 1.0084
Epoch 8/10
64/64 [==============================] - 322s 5s/step - loss: 0.9330
Epoch 9/10
64/64 [==============================] - 326s 5s/step - loss: 0.8741
Epoch 10/10
64/64 [==============================] - 331s 5s/step - loss: 0.8260


In [40]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [42]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            23296     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 91)             93275     
Total params: 4,054,875
Trainable params: 4,054,875
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [44]:
print(generate_text(model, start_string=u"The government: "))

The government: 



(a) 	the production, inform the application line, within unlograph 2; and 
 by agreed in the cisitions on a financial 
westor or administrative laws and registration shall established im non-originating good; 



(h) 	the term “commercial advantageshig on theiral measure shall frem tenderives 
And regulations of conformity would branch on 
importation of, or maintononogication. 



5. Each Party shall provide the importing Party comple where the committee on Japan:  (a) Article 178 shall not a deveation or meental additional implementation and operation to change before fuch be diblogage, ex may be amended (ancerfidentice, the Parties provided, the Party whrough that to 
package 
claimed, markets adoplad et not exceed 
a Party;  (b) that such 
tender of 
publicly or recredunct 
waysomments are imposes on services or central be confidenting, which aro nes in at least 10 perform to ensources and committee, conclude a ProOment point where appointed 
by mialing and acti

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [48]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.510489463806152
Epoch 1 Loss 2.4011
Time taken for 1 epoch 320.8266124725342 sec

Epoch 2 Batch 0 Loss 2.4621353149414062
Epoch 2 Loss 2.0178
Time taken for 1 epoch 315.34421396255493 sec

Epoch 3 Batch 0 Loss 2.0139846801757812
Epoch 3 Loss 1.6677
Time taken for 1 epoch 317.2058184146881 sec

Epoch 4 Batch 0 Loss 1.6108020544052124
Epoch 4 Loss 1.4069
Time taken for 1 epoch 317.913449048996 sec

Epoch 5 Batch 0 Loss 1.3985371589660645
Epoch 5 Loss 1.2164
Time taken for 1 epoch 325.3356149196625 sec

Epoch 6 Batch 0 Loss 1.1611990928649902
Epoch 6 Loss 1.0482
Time taken for 1 epoch 335.0759530067444 sec

Epoch 7 Batch 0 Loss 1.0238412618637085
Epoch 7 Loss 0.9451
Time taken for 1 epoch 335.58014822006226 sec

Epoch 8 Batch 0 Loss 0.9054279923439026


KeyboardInterrupt: ignored